# SDE Weathering Demo

Apply an OU process to a sphere using VIX data.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42)


from src.noise.data_sources import VolatilityFetcher
from src.sde.solvers import apply_sde_to_points, create_sphere, get_trajectory
from src.viz.weathering import color_by_displacement, create_weathering_animation, save_point_cloud

SMOKE_TEST = True

results_dir = Path('results')
results_dir.mkdir(exist_ok=True)

if SMOKE_TEST:
    start, end = '2023-01-01', '2024-01-01'
else:
    start, end = '2014-01-01', '2024-01-01'

fetcher = VolatilityFetcher(start=start, end=end)
series = fetcher.fetch()
sigma_t = fetcher.to_diffusion_coefficients(scale=0.02)

series.plot(title='VIX Volatility')
plt.show()

sphere = create_sphere(n_points=2000, radius=1.0)
trajectory = get_trajectory(sphere, sigma_t=sigma_t, theta=0.1, mu_mode='centroid', seed=42)
weathered = trajectory[-1]
colors = color_by_displacement(sphere, weathered)

output_path = results_dir / 'weathered_sphere.ply'
save_point_cloud(weathered, output_path.as_posix(), colors=colors)
print(f'Saved point cloud to {output_path}')

animation_path = results_dir / 'weathering.gif'
try:
    create_weathering_animation(trajectory, animation_path.as_posix(), fps=10)
    print(f'Saved animation to {animation_path}')
except Exception as exc:
    print(f'Animation skipped: {exc}')
